In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
# import datasets
df_matches = pd.read_csv("WorldCupMatches.csv")
df_players = pd.read_csv("WorldCupPlayers.csv")
df_world_cups = pd.read_csv("WorldCups.csv")

# 1.0 Data Preprocessing

## df_matches

In [11]:
df_matches.describe().round(2)

,Year,Home Team Goals,Away Team Goals,Attendance,Half-time Home Goals,Half-time Away Goals,RoundID,MatchID
count,852.00,852.00,852.00,850.00,852.00,852.00,852.00,8.520000e+02
mean,1985.09,1.81,1.02,45164.80,0.71,0.43,10661772.59,6.134687e+07
std,22.45,1.61,1.09,23485.25,0.94,0.69,27296131.70,1.110572e+08
min,1930.00,0.00,0.00,2000.00,0.00,0.00,201.00,2.500000e+01
25%,1970.00,1.00,0.00,30000.00,0.00,0.00,262.00,1.188750e+03
50%,1990.00,2.00,1.00,41579.50,0.00,0.00,337.00,2.191000e+03
75%,2002.00,3.00,2.00,61374.50,1.00,1.00,249722.00,4.395006e+07
max,2014.00,10.00,7.00,173850.00,6.00,5.00,97410600.00,3.001865e+08


In [10]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  852 non-null    float64
 1   Datetime              852 non-null    object 
 2   Stage                 852 non-null    object 
 3   Stadium               852 non-null    object 
 4   City                  852 non-null    object 
 5   Home Team Name        852 non-null    object 
 6   Home Team Goals       852 non-null    float64
 7   Away Team Goals       852 non-null    float64
 8   Away Team Name        852 non-null    object 
 9   Win conditions        852 non-null    object 
 10  Attendance            850 non-null    float64
 11  Half-time Home Goals  852 non-null    float64
 12  Half-time Away Goals  852 non-null    float64
 13  Referee               852 non-null    object 
 14  Assistant 1           852 non-null    object 
 15  Assistant 2          

## df_players

In [12]:
df_players.describe().round(2)

,RoundID,MatchID,Shirt Number
count,37784.00,3.778400e+04,37784.00
mean,11056474.45,6.362233e+07,10.73
std,27701436.53,1.123916e+08,6.96
min,201.00,2.500000e+01,0.00
25%,263.00,1.199000e+03,5.00
50%,337.00,2.216000e+03,11.00
75%,255931.00,9.741000e+07,17.00
max,97410600.00,3.001865e+08,23.00


In [13]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   RoundID        37784 non-null  int64 
 1   MatchID        37784 non-null  int64 
 2   Team Initials  37784 non-null  object
 3   Coach Name     37784 non-null  object
 4   Line-up        37784 non-null  object
 5   Shirt Number   37784 non-null  int64 
 6   Player Name    37784 non-null  object
 7   Position       4143 non-null   object
 8   Event          9069 non-null   object
dtypes: int64(3), object(6)
memory usage: 2.6+ MB


## df_world_cups

In [16]:
df_world_cups.describe().round(2)

,Year,GoalsScored,QualifiedTeams,MatchesPlayed
count,20.00,20.00,20.00,20.00
mean,1974.80,118.95,21.25,41.80
std,25.58,32.97,7.27,17.22
min,1930.00,70.00,13.00,17.00
25%,1957.00,89.00,16.00,30.50
50%,1976.00,120.50,16.00,38.00
75%,1995.00,145.25,26.00,55.00
max,2014.00,171.00,32.00,64.00


In [17]:
df_world_cups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            20 non-null     int64 
 1   Country         20 non-null     object
 2   Winner          20 non-null     object
 3   Runners-Up      20 non-null     object
 4   Third           20 non-null     object
 5   Fourth          20 non-null     object
 6   GoalsScored     20 non-null     int64 
 7   QualifiedTeams  20 non-null     int64 
 8   MatchesPlayed   20 non-null     int64 
 9   Attendance      20 non-null     object
dtypes: int64(4), object(6)
memory usage: 1.7+ KB


In [20]:
# print the winners of each world cup by counting the number of times each country appears in the winner column
df_world_cups["Winner"].value_counts()

Brazil        5
Italy         4
Germany FR    3
Uruguay       2
Argentina     2
England       1
France        1
Spain         1
Germany       1
Name: Winner, dtype: int64